<!--NAVIGATION-->
< [Introduction](Introduction.ipynb) | [Contenus](Contents.ipynb) | [Condition 1](Condition_1.ipynb) >

# 2. Préparation
## 2.1 Introduction
Pour cette tâche, vous disposez des couches raster suivantes: `buildg.tif`, `roads.tif`, `dtm.tif`, et `gwlevel.tif`. Vous pouvez trouver ces fichiers dans le dossier `data`.

Pour travailler avec PCRaster, nous devons convertir les fichiers au format PCRaster. Vous pouvez le faire à partir de la ligne de commande à l’aide des commandes GDAL. Ici, nous allons faire la conversion avec la bibliothèque Python GDAL.

## 2.2 Convertir des rasters avec GDAL

Convertissons le fichier `buildg.tif`. 
D’abord nous importons `gdal` et `gdalconst` depuis la bibliothèque `osgeo`:

In [ ]:
from osgeo import gdal, gdalconst

Ensuite nous lisons le GeoTIFF depuis notre dossier `data`.

In [ ]:
src_ds = gdal.Open( "data/buildg.tif" )

Ensuite, nous pouvons utiliser l’équivalent Python de [`gdal_translate`](https://gdal.org/programs/gdal_translate.html#gdal-translate): `Translate`. Nous devons utiliser le nom du format. See [GDAL Raster drivers (nom court)](https://gdal.org/drivers/raster/index.html) pour formats. Dans notre cas, le format de sortie est `PCRaster`.

Nous devons également définir le type de données de la sortie.

PCRaster utilise le typage de données dans la base de données: chaque carte a l’un des six types de données utilisés qui lui est attaché. Ces types de données vous aident, vous et PCRaster, à structurer les données. Voir le tableau ci-dessous.<br>

| type de donné| description d’attributs | domaine | exemple |
|:-------:|:-------:|:-------:|:-------:|
| booléen | booléen | 0 (FALSE),<br> 1 (TRUE) | convenable/non convenable, visible/non visible |
| nominal | classifié, non ordonné | toutes les valeurs | classes de sols |
| ordinal | classifié, ordonné | toutes les valeurs | succession d’états, groupes de revenus |
| scalaire | continu, linéaire | valeurs réelles | température, concentration |
| directionnel | continu, directionnel | 0 à 360 dégrés | aspect |
| ldd | direction vers la cellule voisine | codes de directions | réseaux de drainage |

Dans les arguments de `Translate`, vous devez définir le `outputType` et `metadataOptions` en fonction du tableau ci-dessous. Notez que nous ne pouvons convertir que les rasters booléens, nominaux et scalaires. Les rasters ldd et directionnels sont créés dans PCRaster. Pour scalaire, nous pouvons utiliser Float 32 ou Float 64, en fonction de la précision souhaitée. 

| type de données | `outputType` | `metadataOptions` |
|:-------:|:-------:|:-------:|
| booléen | `gdalconst.GDT_Byte` | `'VS_BOOLEAN'`|
| nominal | `gdalconst.GDT_Int32` | `'VS_NOMINAL'`|
| scalaire | `gdalconst.GDT_Float32` | `'VS_SCALAR'`|
| scalaire | `gdalconst.GDT_Float64` | `'VS_SCALAR'`|

`buildg.tif` est un raster nominal. Il contient des classes. Ainsi le code que nous allons utiliser pour `Translate` est:

In [ ]:
dst_ds = gdal.Translate('data/buildg.map', src_ds, format='PCRaster', \
                        outputType=gdalconst.GDT_Int32, metadataOptions='VS_NOMINAL')

Et nous fermons correctement les jeux de données à vider sur le disque:

In [ ]:
dst_ds = None
src_ds = None

Maintenant, vous pouvez trouver le fichier `buildg.map` sur votre disque dur. Vérifiez si c’est le cas si vous exécutez ce notebook localement.

Appliquons le même code aux autres fichiers `dtm.tif`, `roads.tif` et `gwlevel.tif`. Dans ce cas, il est plus facile de l’écrire en tant que fonction. 

In [ ]:
#Importez gdal
from osgeo import gdal

def ConvertToPCRaster(src_filename,dst_filename,ot,VS):
    #Ouvrir le jeu de données existant
    src_ds = gdal.Open(src_filename)
    
    #Traduire GDAL
    dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    #Fermez correctement les jeux de données à vider sur le disque
    dst_ds = None
    src_ds = None
    
ConvertToPCRaster("data/buildg.tif","data/buildg.map",gdalconst.GDT_Int32,"VS_NOMINAL")
ConvertToPCRaster("data/roads.tif","data/roads.map",gdalconst.GDT_Int32,"VS_NOMINAL")
ConvertToPCRaster("data/gwlevel.tif","data/gwlevel.map",gdalconst.GDT_Float32,"VS_SCALAR")
ConvertToPCRaster("data/dtm.tif","data/dtm.map",gdalconst.GDT_Float32,"VS_SCALAR")

## 2.3 Inspection des données
Pour l’algèbre cartographique, les propriétés de toutes les couches raster utilisées dans les calculs doivent être les mêmes.
Voyons s’ils ont le même nombre de lignes et de colonnes, de coordonnées et de taille de pixels.

Ouvrons un fichier PCRaster avec GDAL.

In [ ]:
RasterLayer = gdal.Open("data/dtm.map")

Pour obtenir le nombre de lignes et de colonnes, nous pouvons utiliser les fonctions GDAL `RasterXSize` et `RasterYSize`.
`RasterLayer.GetDescription()`donne le chemin relatif de la couche.

In [ ]:
Path = RasterLayer.GetDescription()
NrColumns = RasterLayer.RasterXSize
NrRows = RasterLayer.RasterYSize
print('{} has {} columns and {} rows'.format(Path,NrColumns,NrRows ))

Pour connaître les coordonnées et la taille des pixels, nous pouvons utiliser `GetGeoTransform` de GDAL.
Il renvoie un tuple avec les informations suivantes:
`RasterLayer.GetGeoTransform[0]`: Coordonnée X en haut à gauche

`RasterLayer.GetGeoTransform[1]`: Résolution des pixels Ouest-Est

`RasterLayer.GetGeoTransform[2]`: Rotation, 0 pour "nord vers le haut"

`RasterLayer.GetGeoTransform[3]`: Coordonnée Y en haut à gauche

`RasterLayer.GetGeoTransform[4]`: Rotation, 0 pour "nord vers le haut"

`RasterLayer.GetGeoTransform[5]`: Résolution des pixels Nord-Sud (négatif)

In [ ]:
RasterLayer.GetGeoTransform()

Présentons cela plus lisible.

In [ ]:
print("Origin = ({}, {})".format(RasterLayer.GetGeoTransform()[0], RasterLayer.GetGeoTransform()[3]))
print("Pixel Size = ({}, {})".format(RasterLayer.GetGeoTransform()[1],RasterLayer.GetGeoTransform()[5]))

Nous pouvons également utiliser GDAL pour obtenir les informations de projection. Le format PCRaster ne stocke pas les informations de projection, mais lorsque nous convertissons les rasters, il enregistre un fichier XML avec les informations de projection. La fonction GDAL est `GetProjection()`.

In [ ]:
RasterLayer.GetProjection()

Cela nous donne une chaîne au format OGC WKT.
Afin de rendre la projection et les unités lisibles, nous devons analyser la chaîne OGC WKT. Pour cela, nous utiliserons la bibliothèque [PyCRS](https://pypi.org/project/PyCRS/). 

S’il n’est pas encore installé si vous exécutez ce notebook localement, vous pouvez installer la bibliothèque en tapant ce qui suit dans l’invite Anaconda:

`conda install -c conda-forge pycrs`

Tout d’abord, nous importons la bibliothèque et enregistrons la chaîne OGC WKT dans une variable que nous analysons.

In [ ]:
import pycrs
RasterLayerProjection = RasterLayer.GetProjection()

# Analyser la chaîne OGC WKT 
crs = pycrs.parse.from_ogc_wkt(RasterLayerProjection)

Nous pouvons vérifier s’il s’agit d’un système de coordonnées projeté:

In [ ]:
isinstance(crs, pycrs.ProjCS)

Le résultat est `True`, nous n’avons donc pas affaire à un système de coordonnées géographiques. Prenons le nom de la projection.

In [ ]:
ProjectionName = crs.name
print(ProjectionName)

Nous pouvons également obtenir les unités de la projection:

In [ ]:
ProjectionUnits = crs.unit.unitname.ogc_wkt
print(ProjectionUnits)

Enfin, il serait utile d’obtenir quelques statistiques de la couche raster. Calculons la valeur minimale et maximale.
Étant donné que les couches raster peuvent avoir plusieurs bandes, nous devons sélectionner la bande. Dans notre cas, nous avons une couche raster à bande unique, nous devons donc choisir la bande 1.

In [ ]:
RasterLayerBand = RasterLayer.GetRasterBand(1)

Nous pouvons maintenant calculer les valeurs minimale et maximale de la bande en utilisant respectivement `GetMinimum` et `GetMaximum`.

In [ ]:
RasterLayerMinimum = RasterLayerBand.GetMinimum()
RasterLayerMaximum = RasterLayerBand.GetMaximum()
print("Minimum: ", RasterLayerMinimum)
print("Maximum: ", RasterLayerMaximum)

Écrivons ceci dans une fonction et obtenons les propriétés de couche raster pour toutes les couches raster afin que nous puissions comparer.

In [ ]:
from osgeo import gdal
import pycrs

def RasterLayerProperties(RasterLayer):
    print("Raster file: {}".format(RasterLayer.GetDescription()))
    print("Driver: {}/{}".format(RasterLayer.GetDriver().ShortName,
                            RasterLayer.GetDriver().LongName))
    print("Size is {} x {} x {}".format(RasterLayer.RasterXSize,
                                    RasterLayer.RasterYSize,
                                    RasterLayer.RasterCount))
    RasterLayerProjection = RasterLayer.GetProjection()
    crs = pycrs.parse.from_ogc_wkt(RasterLayerProjection)
    print("Projection:",crs.name)
    print("Map units:",crs.unit.unitname.ogc_wkt)
    geotransform = RasterLayer.GetGeoTransform()
    if geotransform:
        print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
        print("Pixel Size = ({} {}, {} {})".format(geotransform[1],crs.unit.unitname.ogc_wkt, \
                                                   geotransform[5],crs.unit.unitname.ogc_wkt))
    RasterLayerBand = RasterLayer.GetRasterBand(1)
    print("Minimum: {}".format(RasterLayerBand.GetMinimum()))
    print("Maximum: {}".format(RasterLayerBand.GetMaximum()))
    
    print()
    RasterLayer = None
    
    
DTMLayer = gdal.Open( "data/dtm.map" )
RasterLayerProperties(DTMLayer)

BuildgLayer = gdal.Open( "data/buildg.map" )
RasterLayerProperties(BuildgLayer)

RoadsLayer = gdal.Open( "data/roads.map" )
RasterLayerProperties(RoadsLayer)

GWLevelLayer = gdal.Open( "data/gwlevel.map" )
RasterLayerProperties(GWLevelLayer)

Sur la base de ces résultats, nous pouvons conclure que les couches raster ont les mêmes propriétés et conviennent à l’algèbre cartographique.

<!--NAVIGATION-->
< [Introduction](Introduction.ipynb) | [Contenus](Contents.ipynb) | [Condition 1](Condition_1.ipynb) >